In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [8]:
######################## libraries ########################

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ticker
import numpy as np

import sys
sys.path.append('../../')

from utils.utility_functions import label_top
from utils.utility_functions import get_last_name
from utils.utility_functions import make_dash_zero

##################### CHARTING OPTIONS #####################

# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Palatino']})

rc('text', usetex=True)
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

colors = ['#8a170f', '#8a540f', '#828a0f', '#458a0f', '#0f8a17', '#0f8a54', '#0f828a', '#0f458a', '#170f8a', '#540f8a', '#8a0f82', '#8a0f45']


##################### SOURCES OPTIONS #####################

archival_sources = ['Metropolitan Opera', 'Opera Scotland', 'Royal Opera House', 'Teatro alla Scala', 'Wiener Staatsoper']
commercial_sources_world = ['bachtrack.com','operabase.com','operacritic.com']
commercial_sources_uk = ['Concert-Diary','U.K. Theatre Web']
government_sources = ['National Central Opera Service Reports', 'Opera America']

##############################################################################

In [129]:
import pandas as pd

columns=['source_id', 'source_name', 'composer', 'work', 'country', 'state', 'city', 'theatre', 'date_start', 'date_end', 'premiere_date', 'performance_count']

cursor = conn.cursor()  
query = "SELECT s.id as source_id, " + \
        "       s.name as source_name, " + \
        "       coalesce(c.name, l.composer) as composer, " + \
        "       coalesce(w.name, l.work) as work, " + \
        "       l.country, " + \
        "       l.state, " + \
        "       l.city, " + \
        "       l.theatre, " + \
        "       l.date_start, " + \
        "       l.date_end, " + \
        "       w.premiere_date, " + \
        "       l.performance_count " + \
        "from sources s " + \
        "INNER JOIN listings l on s.id = l.source_id " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "LEFT JOIN works w ON lm.work_uri = w.uri " + \
        "LEFT JOIN composers c ON w.composer_id = c.id " + \
        "ORDER BY s.id, l.country, l.state, l.city"
        
cursor.execute(query)

mdf = pd.DataFrame(cursor.fetchall(), columns=columns)
mdf['year'] = mdf.apply(lambda row: row['date_start'].year, axis=1)
mdf['decade'] = mdf.apply(lambda row: 10*int(row['year']/10), axis=1)
mdf[['premiere_date']] = mdf[['premiere_date']].fillna(value=0)
mdf['work_decade'] = mdf.apply(lambda row: 10*int(row['premiere_date']/10), axis=1)

In [130]:
######################## conservativeness amd innovation ########################
sources = archival_sources

o_df = mdf.copy()
o_df = o_df[o_df['source_name'].isin(sources)]
o_df = o_df[o_df['work_decade']!=0]
o_df = o_df[o_df['year']<2019]

o_df['innovation'] = o_df.apply(lambda row: row['year']- row['premiere_date'], axis=1)
o_df = o_df.sort_values(by=['innovation'])
o_df = o_df[o_df['innovation'] < 0]

o_df

,source_id,source_name,composer,work,country,state,city,theatre,date_start,date_end,premiere_date,performance_count,year,decade,work_decade,innovation
142139,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,England,London,"Hub, The",2009-03-07,2009-03-07,2014.0,1,2009,2000,2010,-5.0
142243,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,England,London,"Hub, The",2009-03-08,2009-03-08,2014.0,1,2009,2000,2010,-5.0
148717,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,Scotland,Glasgow,Òran Mór,2009-02-20,2009-02-20,2014.0,1,2009,2000,2010,-5.0
148897,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,Scotland,Glasgow,Òran Mór,2009-02-21,2009-02-21,2014.0,1,2009,2000,2010,-5.0
142121,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,England,London,"Hub, The",2009-03-07,2009-03-07,2014.0,1,2009,2000,2010,-5.0
148898,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,Scotland,Glasgow,Òran Mór,2009-02-21,2009-02-21,2014.0,1,2009,2000,2010,-5.0
148896,7,Opera Scotland,Mathis Nitschke,Happy Happy,United Kingdom,Scotland,Glasgow,Òran Mór,2009-02-22,2009-02-22,2014.0,1,2009,2000,2010,-5.0
149173,7,Opera Scotland,Stuart MacRae,The Devil Inside,United Kingdom,Scotland,Glasgow,"Theatre Royal, Glasgow",2015-01-23,2015-01-23,2016.0,1,2015,2010,2010,-1.0


In [122]:
prd = mdf[(mdf['source_name']=='Teatro alla Scala')*(mdf['year']==1982) ]
prd = prd.sort_values(by=['premiere_date'], ascending=False)
prd

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


,source_id,source_name,composer,work,country,state,city,theatre,date_start,date_end,premiere_date,performance_count,year,decade,work_decade
313691,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-20,1982-09-20,1982.0,1,1982,1980,1980
313722,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-24,1982-09-24,1982.0,1,1982,1980,1980
313720,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-24,1982-09-24,1982.0,1,1982,1980,1980
313719,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-23,1982-09-23,1982.0,1,1982,1980,1980
313718,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-23,1982-09-23,1982.0,1,1982,1980,1980
313717,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-23,1982-09-23,1982.0,1,1982,1980,1980
313716,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-22,1982-09-22,1982.0,1,1982,1980,1980
313714,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-22,1982-09-22,1982.0,1,1982,1980,1980
313713,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-21,1982-09-21,1982.0,1,1982,1980,1980
313712,10,Teatro alla Scala,Luca Mosca,Il sogno di Titania,Italy,None,Milan,Teatro alla Scala,1982-09-21,1982-09-21,1982.0,1,1982,1980,1980


In [118]:
c_df

,source_name,year,total_age_works,work,premiere_date,conservativeness,innovation
